In [2]:
import kivy
from kivy.app import App 
from kivy.uix.label import Label
from kivy.uix.gridlayout import GridLayout
from kivy.uix.textinput import TextInput
from kivy.uix.button import Button
from kivy.uix.togglebutton import ToggleButton
from kivy.uix.widget import Widget
from kivy.properties import ObjectProperty
from kivy.uix.floatlayout import FloatLayout
from kivy.graphics import Rectangle
from kivy.graphics import Color
from kivy.graphics import Line
from kivy.lang import Builder
from kivy.uix.screenmanager import ScreenManager, Screen
from kivy.core.window import Window
from kivy.properties import StringProperty
from kivy.properties import NumericProperty
from kivy.properties import ListProperty

Window.size = (800, 800)

calc_list = [0] * 8
print(calc_list)

class ConnectionWindow(Screen):

    #Save Connection Type Chosen
    connection = ObjectProperty(None)
    def connection_fixed(self):
        connection = "fixed"
        print(connection)
        ConnectionWindow.connection = connection
        calc_list [0] = connection
    def connection_pin (self):
        connection = "pin"
        print(connection)
        ConnectionWindow.connection = connection
        calc_list [0] = connection


class VariablesWindow (Screen):
    
    #Save Load Type Chosen
    load = ObjectProperty(None)
    def load_distributed(self):
        load = "distributed"
        VariablesWindow.load = load
        calc_list [1] = load
    def load_point(self):
        load = "point"
        VariablesWindow.load = load
        calc_list [1] = load
    def load_moment(self):
        load = "moment"
        VariablesWindow.load = load
        calc_list [1] = load   
    # def save_load_type(self):
    #     print("Load Type Saved")
    #     print(self.load)
    #     print("Connection Type Saved")
    #     print(calc_list)

    #Save Rest of Variables
    # container = ObjectProperty(None)
    # def save_variables(self):
    #     for child in reversed(self.container.children):
    #         if isinstance(child, TextInput):
    #             self.calc_list.append(child.text)

    def save_variables(self):
        calc_list [2] = load_value = self.ids.load_value.text
        calc_list [3] = a_value = self.ids.a_value.text
        calc_list [4] = r_value = self.ids.r_value.text
        calc_list [5] = b_value = self.ids.b_value.text
        calc_list [6] = h_value = self.ids.h_value.text
        calc_list [7] = l_value = self.ids.l_value.text
        print(calc_list)
        

class OutputWindow (Screen):
    pass

class WindowManager(ScreenManager):
    load = StringProperty("")
    connection = StringProperty("")
    pass

kv = Builder.load_file("my.kv")

class MyApp (App):
    def build (self):
        return kv

#Run Program 
if __name__ == "__main__":
    MyApp().run()



[0, 0, 0, 0, 0, 0, 0, 0]
fixed
fixed
fixed
['fixed', 'moment', '', '', '', '', '', '']
['fixed', 'distributed', '', '', '', '', '', '']
['fixed', 'point', '', '', '', '', '', '']
['fixed', 'moment', '', '', '', '', '', '']
